In [12]:
import re
import openai
import toolmode.data.funcqa as funcqa_dataset
import importlib

In [2]:
importlib.reload(funcqa_dataset)

dataset = funcqa_dataset.load()

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.89it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'answer_number'],
        num_rows: 650
    })
    test: Dataset({
        features: ['question', 'answer_number', 'answer'],
        num_rows: 68
    })
})

Our dataset contains only multihop questions

In [9]:
max(map(lambda x: len(re.findall("<T>", x)), dataset["train"]["answer"]))

1